# 2.3.3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* Best result: A2

In [8]:
from utils.parser import Parser
from utils.openai import OpenAI
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()

In [9]:
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print(len(pdf_prompts))

66


### Art der baulichen Nutzung

In [ ]:
context = parser.text2prompts(['''
Hintergrundwissen zum Thema "Art der baulichen Nutzung":
- § 1 Allgemeine Vorschriften für Bauflächen und Baugebiete
- § 2 Kleinsiedlungsgebiete
- § 3 Reine Wohngebiete
- § 4 Allgemeine Wohngebiete
- § 4a Gebiete zur Erhaltung und Entwicklung der Wohnnutzung (besondere Wohngebiete)
- § 5 Dorfgebiete
- § 5a Dörfliche Wohngebiete
- § 6 Mischgebiete
- § 6a Urbane Gebiete
- § 7 Kerngebiete
- § 8 Gewerbegebiete
- § 9 Industriegebiete
- § 10 Sondergebiete, die der Erholung dienen
- § 11 Sonstige Sondergebiete
- § 12 Stellplätze und Garagen
- § 13 Gebäude und Räume für freie Berufe
- § 13a Ferienwohnungen
- § 14 Nebenanlagen; Anlagen zur Nutzung solarer Strahlungsenergie und Kraft-Wärme-Kopplungsanlagen
- § 15 Allgemeine Voraussetzungen für die Zulässigkeit baulicher und sonstiger Anlagen
'''])

In [11]:
# A.1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'

messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung: Informationen

#### § 4 Art der baulichen Nutzung
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

Zulässig sind:
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

Nicht zulässig sind:
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

#### D.5.2.1.1. Allgemeines Wohngebiet
Nach dem aktuellen Planungskonzept wird vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebiets vorgesehen. Die Festsetzungen zu den im Planbereich ausgewiesenen Wohnnutzungen orientieren sich dabei am Nutzungskatalog der allgemeinen Wohngebiete gemäß § 

In [13]:
# A.2) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT (BEST RESULT)
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Art der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'

msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
pprint(msg_pages)
pages = [6, 7, 26, 27, 28] #msg_pages
relevant_pages = list(map(lambda index: pdf_prompts[index-1], pages))
pprint(len(relevant_pages))

samantha.addContext(context)

msg233_art = samantha.request([
    *relevant_pages,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
    },
])

samantha.clearContext()

pprint(msg233_art)
%store msg233_art

```json
{
  "Seiten": [6, 7, 26, 27, 28]
}
```
#############################################


### Alle Themen in einem Prompt

In [ ]:
# D.1) Textinformationen – WITH CONTEXT
# Schema / BauNVO
%store -r msgbaunvo
schema = '''
+----------------------------------------------+
| Art der baulichen Nutzung                    |
+----------------------------------------------+
| Grundflächenzahl (GRZ) | Geschossfläche (GF) |
+----------------------------------------------+
'''
context = parser.text2prompts([msgbaunvo, schema])
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
pprint(messages)

In [3]:
# D.2) Ein Thread – Analyse und Extraktion in einem Schritt – WITHOUT CONTEXT
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'

messages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung:
- Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
- Zulässig sind:
  - Wohngebäude,
  - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe,
  - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.
- Nicht zulässig sind:
  - Betriebe des Beherbergungsgewerbes,
  - sonstige nicht störende Gewerbebetriebe,
  - Anlagen für Verwaltungen,
  - Gartenbaubetriebe,
  - Tankstellen.
- Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung:
- Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
- Zulässige Grundflächenzahl (GRZ) gemäß § 19 Abs. 4 BauNVO:
  - WA1: GRZ maximal 0,65,
  - WA2: GRZ maximal 0,80,
  - WA3: GRZ maximal 0,75,
  - WA4: GRZ maximal 0,75.
- Die zulässigen Gebäudeoberkanten (OK) sind für die einzelnen Bereiche in 

In [5]:
# D.3) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITHOUT CONTEXT
# Wenn Dokument zu groß wird: z.B. chunk_size=10 pages, overlap=2 pages.
# chunk_size = 10, overlap = 2
# n = math.ceil(len(pdf_prompts)/chunk_size)
# chunks = list(map(lambda i: pdf_prompts[slice(max(i*chunk_size-overlap,0),i*chunk_size+chunk_size)], range(n)))
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'

msg_pages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33]
relevant_pages = list(map(lambda index: pdf_prompts[index-1], pages))
pprint(len(relevant_pages))

msg3 = samantha.request([
    *relevant_pages,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'
    },
])

pprint(msg3)

```json
{
  "Seiten": [6, 7, 8, 28, 29, 30, 31, 32, 33]
}
```


In [7]:
# D.4) Mehrere Threads – WITHOUT CONTEXT
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def run(pdf_prompt):
    return samantha.request([
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
        },
        pdf_prompt
    ])

prompt_chain = list(map(lambda pdf_prompt: run(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)
for result in results:
    pprint(result)

Art der baulichen Nutzung: -
Maß der baulichen Nutzung: -
Bauweise: -
überbaubare Grundstücksfläche: -
Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Inhaltsverzeichnis des Bebauungsplans Nr. 872 A, Aufstellung:

- **Art der baulichen Nutzung**: -
- **Maß der baulichen Nutzung**: -
- **Bauweise**: -
- **Überbaubare Grundstücksfläche**: -

Das Inhaltsverzeichnis enthält keine spezifischen Informationen zu diesen Themen. Um detaillierte Informationen zu erhalten, müsste der vollständige Text des Bebauungsplans eingesehen werden.
Die Seite enthält nur eine Liste von Abkürzungen und deren Erläuterungen. Es sind keine Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche" vorhanden.

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
- Art der baulichen Nutzung: WA (Allgemeines Wohngebiet)
- Maß der baulichen Nutzung: -
- Bau